In [ ]:
# ✅ Análise Ambiental com Dados do Google Drive (SEMA-MT)
#
# Este notebook acessa os arquivos diretamente da pasta:
# /Meu Drive/Ia_legislacao/Bases/
#
# Arquivos esperados:
# - VEGETACAO_RADAMBRASIL.zip
# - USO_CONSOLIDADO.zip
# - UNIDADES_CONSERVACAO.zip
# - TERRAS_INDIGENAS.zip
# - LIM_MUNICIPIOS_MT.zip
#
# 🚨 O usuário só precisa fazer o upload de ATP.zip (shape da fazenda)


In [ ]:
!pip install geopandas shapely matplotlib -q

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import zipfile, os
from google.colab import drive, files
from shapely.geometry import Polygon, MultiPolygon
from matplotlib.patches import Polygon as MplPolygon
from matplotlib.collections import PatchCollection

In [ ]:
drive.mount('/content/drive')
base_dir = "/content/drive/MyDrive/Ia_legislacao/Bases/"

In [ ]:
def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def find_shapefile(folder_path):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".shp"):
                return os.path.join(root, file)
    return None

In [ ]:
print("📂 Envie o arquivo ATP.zip (shapefile da fazenda)...")
uploaded = files.upload()
unzip_file("ATP.zip", "ATP")

In [ ]:
unzip_file(os.path.join(base_dir, 'VEGETACAO_RADAMBRASIL.zip'), 'RADAM')
unzip_file(os.path.join(base_dir, 'USO_CONSOLIDADO.zip'), 'USO')
unzip_file(os.path.join(base_dir, 'UNIDADES_CONSERVACAO.zip'), 'UC')
unzip_file(os.path.join(base_dir, 'TERRAS_INDIGENAS.zip'), 'TI')
unzip_file(os.path.join(base_dir, 'LIM_MUNICIPIOS_MT.zip'), 'MUN')

In [ ]:
fazenda = gpd.read_file(find_shapefile('ATP')).to_crs("EPSG:32721")
radam = gpd.read_file(find_shapefile('RADAM')).to_crs("EPSG:32721")
uso = gpd.read_file(find_shapefile('USO')).to_crs("EPSG:32721")
uc = gpd.read_file(find_shapefile('UC')).to_crs("EPSG:32721")
ti = gpd.read_file(find_shapefile('TI')).to_crs("EPSG:32721")
mun = gpd.read_file(find_shapefile('MUN')).to_crs("EPSG:32721")

In [ ]:
def cor_por_fitoecolog(nome):
    nome = nome.lower()
    if 'floresta' in nome:
        return '#336633'
    elif 'cerrado' in nome:
        return '#f3e1b4'
    else:
        return '#dddddd' 

In [ ]:
radam_clip = radam[radam.intersects(fazenda.unary_union)]
intersec_radam = gpd.overlay(radam_clip, fazenda, how='intersection')
intersec_radam = intersec_radam[intersec_radam.is_valid & ~intersec_radam.is_empty]
intersec_radam['cor'] = intersec_radam['FITOECOLOG'].apply(cor_por_fitoecolog)
intersec_radam['area_ha'] = intersec_radam.geometry.area / 10000
intersec_radam['buffer'] = intersec_radam.buffer(50)

fig, ax = plt.subplots(figsize=(12, 12))
fazenda.boundary.plot(ax=ax, color='black', linewidth=2.5)
for _, row in intersec_radam.iterrows():
    color = row['cor']
    geom = row['buffer']
    if isinstance(geom, Polygon):
        patch = MplPolygon(list(geom.exterior.coords))
        ax.add_patch(patch)
        patch.set_facecolor(color)
    elif isinstance(geom, MultiPolygon):
        for poly in geom.geoms:
            patch = MplPolygon(list(poly.exterior.coords))
            ax.add_patch(patch)
            patch.set_facecolor(color)
plt.title("Vegetação RADAMBRASIL")
plt.axis('off')
plt.savefig("mapa_radam.jpeg", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
intersec_uso = gpd.overlay(uso, fazenda, how='intersection')
intersec_uso['area_ha'] = intersec_uso.geometry.area / 10000
fig, ax = plt.subplots(figsize=(12, 12))
fazenda.boundary.plot(ax=ax, color='black', linewidth=2.5)
intersec_uso.plot(ax=ax, color='orange', edgecolor='black')
plt.title("Uso Consolidado (SEMA)")
plt.axis('off')
plt.savefig("mapa_uso.jpeg", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
intersec_uc = gpd.overlay(uc, fazenda, how='intersection')
intersec_ti = gpd.overlay(ti, fazenda, how='intersection')
intersec_mun = gpd.overlay(mun, fazenda, how='intersection')
print("🌱 UC:", intersec_uc.area.sum() / 10000, "ha")
print("🧬 TI:", intersec_ti.area.sum() / 10000, "ha")
print("📍 Município:", intersec_mun.area.sum() / 10000, "ha")

In [ ]:
files.download("mapa_radam.jpeg")
files.download("mapa_uso.jpeg")